In [6]:
import numpy as np
import pandas as pd
import scipy as sp
import scipy.linalg
import sympy as sm
from tqdm import tqdm
from ggplot import *

class BaseCalculator:
    def __init__(self, n, gamma):
        self.n = n
        self.gamma = gamma
        self.xn = 0
        self.gen = []
        self.t = sp.stats.norm.ppf((1 + 0.95) / 2)
        self.delta = 0
        
    def calc_xn(self):
        return self.xn
    
    def calc_sigma(self):
        self.sigma = (np.square(self.gen - self.xn).sum() / (self.n - 1)) ** (0.5)
        return self.sigma
    
    def calc_delta(self):
        self.calc_sigma()
        self.delta = self.t * self.sigma / self.n**(1/2)
        return self.delta
    
class Printer:
    def printValues(calculator, n):
        print("")
        for i in range(0, 1):
            print("    Interation {}, n = {}".format(i, n))
            print("Xn = {}".format(calculator.calc_xn()))
            print("Delta = {}".format(calculator.calc_delta()))
            print("Interval = [{} .. {}]".format(calculator.xn - calculator.delta, calculator.xn + calculator.delta))

# Вычисление объема

In [7]:
class VolumeCalculator(BaseCalculator):
    def __init__(self, n):
        super().__init__(n, 0.95)
        self.k = 5
        self.c = 6.85
        self.a = 7
        self.xn = -1

    def f(self, x):
        return np.log(self.a * x + 1) 
    
    def iffer(self, t):
        if t <= self.c:
            return 1
        else:
            return 0
    
    def calc_xn(self):
        self.gen = np.sum(self.f(np.random.uniform(0,1., self.n * self.k).reshape(self.n ,self.k)), axis=1)
        self.gen = np.array([self.iffer(xi) for xi in self.gen])
        self.xn = self.gen.sum() / self.n
        return self.xn

Printer.printValues(VolumeCalculator(10**4), 10**4)
Printer.printValues(VolumeCalculator(10**6), 10**6)



    Interation 0, n = 10000
Xn = 0.4677
Delta = 0.009779839440706454
Interval = [0.4579201605592936 .. 0.47747983944070643]

    Interation 0, n = 1000000
Xn = 0.46705
Delta = 0.0009778522280606803
Interval = [0.46607214777193934 .. 0.4680278522280607]


# Вычисление интеграла $\int_0^\infty x^4e^{-x}dx$

In [8]:
class ExpIntegralCalculator(BaseCalculator):
    def __init__(self, n):
        super().__init__(n, 0.95)
        
    def calc_xn(self):
        self.gen = np.random.exponential(size = self.n) # n values from exponential distribution
        self.gen = np.array([xi**4 for xi in self.gen]) # to power 4
        self.xn = np.sum(self.gen) / self.n # calc expected value
        return self.xn

    
print("Exact value = {}".format(24))
Printer.printValues(ExpIntegralCalculator(10**4), 10**4)
Printer.printValues(ExpIntegralCalculator(10**6), 10**6)


Exact value = 24

    Interation 0, n = 10000
Xn = 22.60200243105615
Delta = 3.1205473219431235
Interval = [19.481455109113025 .. 25.722549752999274]

    Interation 0, n = 1000000
Xn = 23.981765701140716
Delta = 0.3806762311442237
Interval = [23.60108946999649 .. 24.36244193228494]


# Вычисление интеграла $\int_{-\infty}^\infty \sqrt{|x|} \ e^{\frac{-(x+2)^2}{4}}dx$

In [9]:
class NormIntegralCalculator(BaseCalculator):
    def __init__(self, n):
        super().__init__(n, 0.95)
        self.n = n
        self.mu = -2
        self.sigma = 2**(1/2)
        self.value = 0
        
    def calc_xn(self):
        self.gen = np.random.normal(loc = self.mu, scale = self.sigma, size = self.n) # n values from norm(-2, 2) distribution
        self.gen = np.sqrt(np.abs(self.gen)) # sqrt from abs
        self.xn = self.sigma * (2 * np.pi)**(1/2) * np.sum(self.gen) / self.n # calc expected value
        return self.xn
    
    def calc_delta(self):
        return 2 * np.pi**(1/2) * super().calc_delta()

print("Exact value = {}".format(4.8498))
Printer.printValues(NormIntegralCalculator(10**4), 10**4)
Printer.printValues(NormIntegralCalculator(10**6), 10**6)

Exact value = 4.8498

    Interation 0, n = 10000
Xn = 4.853038665829423
Delta = 0.24436315092733657
Interval = [4.784105093651368 .. 4.921972238007479]

    Interation 0, n = 1000000
Xn = 4.850378944206651
Delta = 0.024420823692195125
Interval = [4.843489957032254 .. 4.857267931381047]
